In [ ]:
# Install dependencies (uncomment if needed)
!pip install --upgrade openai gradio pandas matplotlib seaborn rouge-score transformers pymupdf

import openai
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rouge_score import rouge_scorer
from transformers import pipeline
import fitz  # PyMuPDF
import re

# Set your OpenAI API Key
API_KEY = ""  # ⚠️ Make sure to keep this secure
client = openai.OpenAI(api_key=API_KEY)

# Fine-tuned summarizer
finetune_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# GPT Summarizer (still available, though not used in this UI)
def gpt_summarize(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI trained to summarize patents effectively."},
            {"role": "user", "content": f"Summarize the following patent description concisely:\n\n{text}"}
        ],
        max_tokens=200,
        temperature=0.5
    )
    return response.choices[0].message.content.strip()

# Handles file OR pasted text
def process_input(file=None, text_input=None):
    try:
        # Priority to pasted text
        if text_input and text_input.strip():
            desc = text_input.strip()

        # Else try to read PDF
        elif file is not None and file.name.endswith(".pdf"):
            doc = fitz.open(file.name)
            text = "".join([page.get_text() for page in doc])
            description_match = re.search(r"Description:\s*(.*)", text, re.DOTALL)
            desc = description_match.group(1).strip() if description_match else text.strip()

        else:
            return "❌ Please upload a PDF or paste patent text."

        if not desc:
            return "❌ Could not extract meaningful content."

        # Fine-tuned summary
        finetune_summary = finetune_summarizer(desc, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
        return finetune_summary

    except Exception as e:
        return f"❌ Error: {e}"

# Gradio UI: File + Text inputs
iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.File(label="Upload Patent PDF (optional)", type="filepath"),
        gr.Textbox(label="Or Paste Patent Text", lines=10, placeholder="Paste patent content here if no PDF...")
    ],
    outputs=gr.Textbox(label="📄 Fine-Tuned Summary"),
    title="Atuomated Patent Summarizer ",
    description="Upload a patent PDF OR paste the content. This tool will summarize it using a fine-tuned BART model."
)

iface.launch(share=True)


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b3231fc744f231ef27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
